Final Project job list response to request from web app skills posted using in Json via flask framework

In [17]:
# set unit test or live mode for developer
mode = 'unittest'

1. build hard skill ('H') list or soft ('S') skill list based on web page request

In [18]:


# sktype: 'H'ard or 'S'oft skill
# dtype: 'I'D of skill or 'D'escription of skill
def buildsklist(sktype,dtype):
    if sktype == 'H':
        selsk_stmt = '''SELECT Skill_ID, Description FROM Skills WHERE TYPE IN ('cs','ds','pltfm','stat','tech') ORDER BY DESCRIPTION;'''
    else:
        selsk_stmt = '''SELECT Skill_ID, Description FROM Skills WHERE TYPE IN ('ba','domn','nontech') ORDER BY DESCRIPTION;'''

# build skill dictionary {..."2":"Agile",...}
# technical
    import sqlite3
    conn = sqlite3.connect('C:\\DS_FinalProject_Data\\JobSkill.db')
    curs = conn.cursor()
    curs.execute(selsk_stmt)
    skdescs = curs.fetchall()
    
    skmIDs = ''
    skills = ''
    for skid, skdesc in skdescs:
        if skmIDs == '':
            skmIDs = str(skid)
        else:
            skmIDs = skmIDs + ',' + str(skid)
            
        if skills == '':
            skills = skdesc
        else:
            skills = skills + ',' + skdesc
    if dtype == 'I':
        return skmIDs
    else:
        return skills
#
# main
#
import json

if mode != 'unittest':
    sktype = request.get_json(force=True)
else:
    sktype = 'H'
#     sktype = 'S'
skills = buildsklist(sktype,'D')
if sktype == 'H':
    outdict = dict(Hardskills=skills)
else:
    outdict = dict(Softskills=skills)

response = json.dumps(outdict)


2 get request from and post response to web user interface app

In [19]:

from flask import Flask, request, Response, json
import numpy as numpy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

if mode != 'unittest':
    data = request.get_json(force=True)
else:
    skreq = "Python,Statistics,Communication"
    input_data = json.dumps({"skills": skreq})
    data = json.loads(input_data)

requestData = data["skills"]

# Connect to SQLite3 database, select all SkillsJobs rows
import sqlite3
conn = sqlite3.connect('C:\\DS_FinalProject_Data\\JobSkill.db')
curs = conn.cursor()
# Select all SkillJobs rows
sjsel_stmt = '''SELECT SkillsJobs_ID, SkillIDs, JobIDs FROM SkillsJobs;'''
curs.execute(sjsel_stmt)
sjs = curs.fetchall()
conn.commit()

# convert request json to skill IDs list
sks = {}
selsk_stmt = '''SELECT Skill_ID, Description FROM Skills ORDER BY Skill_ID;'''
for skkey, skval in curs.execute(selsk_stmt):
    sks[skkey] = skval
skkey_list = list(sks.keys())
skval_list = list(sks.values())

# get hard and soft skills list
hsklist = buildsklist('H','I')
ssklist = buildsklist('S','I')
hsktcnt = hsklist.count(',')+1
ssktcnt = ssklist.count(',')+1

#  build user input skill list
skids = ''
for skill1 in requestData.split(","):
    position = skval_list.index(skill1.lower())
    skid = skkey_list[position]
    if skids == '':
        skids = str(skid)
    else:
        skids = skids + ',' + str(skid)

# search input skills in each SkillsJobs row, if found, 
# append title, company, location, skills required, % match 
# to response list
# process each SkillsJobs row
jsel_stmt = '''SELECT JobTitle, Company, Location, URL FROM Jobs WHERE Job_ID = ?;'''
outdicts = ''
for sj in sjs:                          # loop through each SkillsJobs, get a skill job record
    sjIDs = sj[0]
    sjskIDs = sj[1]
    sjjobIDs = sj[2]
    thskcnt = 0
    tsskcnt = 0
    tskcnt = 0
    # get job total hard skill, soft skill and all skills count, by loop through each skill on skill job record
    for skid2 in sjskIDs.split(','):
        tskcnt += 1
        for hid2 in hsklist.split(','): 
            if skid2 == hid2: thskcnt += 1
        for sid2 in ssklist.split(','):
            if skid2 == sid2: tsskcnt += 1
    skdesc = ''
# process each input skill
    hskcnt = 0
    sskcnt = 0
    skcnt = 0  
    for skill2 in skids.split(','):          # loop through user skills        
        if skill2 in sjskIDs:                # if user skill in job skills            
            for hsk in hsklist.split(','):   # check if it's a hard skill            
                if skill2 == hsk:
                    hskcnt += 1              # increment user hard skill match count
            for ssk in ssklist.split(','):   # check if it's a soft skill
                if skill2 == ssk:
                    sskcnt += 1
    skcnt = hskcnt+sskcnt                    # total user skill count match with skills on skill job record
    if skcnt > 0:                            # if user skill matches in skills of the skill job record       
        # build job skills desc list
        for skill3 in sjskIDs.split(','):
            skill3 = int(skill3.strip())
            position = skkey_list.index(skill3)
            jsval = skval_list[position]
            if skdesc == '':
                skdesc = jsval
            else:
                skdesc = skdesc+ ','+jsval
        # get title, company, location, url        
        for jobID in sjjobIDs:      
            curs.execute(jsel_stmt, (jobID))
            jrec = curs.fetchone()
            if jrec is not None: 
                title = jrec[0]
                company = jrec[1]
                location = jrec[2]
                url = jrec[3]
                hpercnt = 0; spercent = 0; opercent = 0
                if thskcnt != 0:
                    hpercent = int(hskcnt / thskcnt * 100)
                if tsskcnt != 0:   
                    spercent = int(sskcnt / tsskcnt * 100)
                if tskcnt != 0:
                    opercent = int(skcnt / tskcnt * 100)
                outdict = dict(Title=title, Company=company, Location=location, Skills=skdesc, URL=url, \
                               HPercent=hpercent, SPercent=spercent,OPercent=opercent)
                outline = title+','+company+','+location+',"'+skdesc+'",'+url+','+str(hpercent)\
                                +','+str(spercent)  +','+str(opercent)                             

                if outdicts == '':
                    outdicts = str(outdict)
                else:
                    outdicts = outdicts+','+str(outdict)
response = json.dumps(outdicts)
print(response)

# return Response(json.dumps(outdicts))



"{'Title': 'Data Scientist/Biostatistician', 'Company': 'Scismic', 'Location': 'Palo Alto, CA', 'Skills': 'algorithms,analysis,analytic,aws,cloud,excel,google cloud,machine learning,python,sql,statistics,version control', 'URL': '', 'HPercent': 20, 'SPercent': 0, 'OPercent': 16},{'Title': 'Lead Data Scientist (Remote)', 'Company': 'OBMedia', 'Location': 'Remote', 'Skills': 'analysis,analytic,cloud,communication,data engineering,data science,leadership,machine learning,ml,pipeline ,python,spark,sql', 'URL': '', 'HPercent': 10, 'SPercent': 33, 'OPercent': 15},{'Title': 'Data Scientist III', 'Company': 'MAPFRE', 'Location': 'Boston, MA', 'Skills': 'algorithms,analysis,analytic,data science,leadership,machine learning,math,ml,modeling,predictive,python,spark,sql,statistics,structured and unstructured data', 'URL': '', 'HPercent': 15, 'SPercent': 0, 'OPercent': 13},{'Title': 'Simulation Data Science Engineer', 'Company': 'John Deere', 'Location': 'Moline, IL', 'Skills': 'analysis,analytic,a